In [8]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm
import math
import gc
import os
from sklearn.model_selection import train_test_split

# PART 1: Data Preparation
class IMDBDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def prepare_data(batch_size=2, max_length=256):
    # Load the IMDB dataset
    dataset = load_dataset("stanfordnlp/imdb")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    
    # Split the training data into training and validation manually using train_test_split
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        dataset['train']['text'],  # Use a smaller subset
        dataset['train']['label'],
        test_size=0.2,  # 80% training, 20% validation
        random_state=42
    )
    
    # Prepare training and validation datasets
    train_dataset = IMDBDataset(
        train_texts,
        train_labels,
        tokenizer,
        max_length=max_length
    )
    
    val_dataset = IMDBDataset(
        val_texts,
        val_labels,
        tokenizer,
        max_length=max_length
    )
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
    
    return train_loader, val_loader, tokenizer

# PART 2: Model Definition
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=256):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=64, nhead=4, num_layers=4, dim_feedforward=256, dropout=0.1):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=256)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
            norm_first=True
        )
        
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.classifier = nn.Sequential(
            nn.Linear(d_model, 32),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(32, 2)
        )
        
        self.init_weights()
    
    def init_weights(self):
        initrange = 0.02
        nn.init.uniform_(self.embedding.weight.data, -initrange, initrange)
        for layer in self.classifier:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight.data)
                if layer.bias is not None:
                    nn.init.zeros_(layer.bias.data)
    
    def forward(self, x, attention_mask=None):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        
        if attention_mask is not None:
            attention_mask = attention_mask.bool()
            x = x.masked_fill(~attention_mask.unsqueeze(-1), 0)
        
        x = self.transformer_encoder(x)
        
        if attention_mask is not None:
            mask = attention_mask.unsqueeze(-1).float()
            x = (x * mask).sum(dim=1) / mask.sum(dim=1).clamp(min=1e-9)
        else:
            x = x.mean(dim=1)
        
        x = self.classifier(x)
        return x

# PART 3: Training Procedure
def train_model(batch_size=64, num_epochs=20, learning_rate=0.0001):
    # Prepare data
    train_loader, val_loader, tokenizer = prepare_data(batch_size=batch_size)
    
    # Safe CUDA initialization
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Initialize model and move to device
    model = TransformerClassifier(vocab_size=tokenizer.vocab_size)
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    
    best_val_accuracy = 0
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        model.train()
        
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        for batch in tqdm(train_loader, desc="Training"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = criterion(outputs, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            
            train_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            train_correct += (predictions == labels).sum().item()
            train_total += labels.size(0)
            
            # Clear memory periodically
            if device.type == 'cuda':
                del outputs, loss
                gc.collect()
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = train_correct / train_total
        print(f"Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")
        
        # Validation phase (optional)
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)
                
                outputs = model(input_ids, attention_mask)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                predictions = torch.argmax(outputs, dim=1)
                val_correct += (predictions == labels).sum().item()
                val_total += labels.size(0)
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = val_correct / val_total
        print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    save_path = "transformer_classifier.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")    

if __name__ == "__main__":
    train_model()
    



c:\Users\ashmi\Documents\Artificial Intelligence\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\ashmi\Documents\Artificial Intelligence\.venv\Lib\site-packages\torch\nn\modules\transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Using device: cuda

Epoch 1/20


Training: 100%|██████████| 313/313 [01:17<00:00,  4.02it/s]


Training Loss: 0.7053, Training Accuracy: 0.5078


Validation: 100%|██████████| 79/79 [00:06<00:00, 11.99it/s]


...

Validation Loss: 0.3144, Validation Accuracy: 0.8722

Epoch 20/20


Training: 100%|██████████| 313/313 [01:13<00:00,  4.23it/s]


Training Loss: 0.2663, Training Accuracy: 0.8918


Validation: 100%|██████████| 79/79 [00:06<00:00, 13.17it/s]

Validation Loss: 0.4746, Validation Accuracy: 0.8210
Model saved to transformer_classifier.pth


In [13]:
import torch
from transformers import AutoTokenizer

# Load the tokenizer and model configuration
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and load the saved state dictionary
model = TransformerClassifier(vocab_size=tokenizer.vocab_size)
model.load_state_dict(torch.load("transformer_classifier.pth", map_location=device))
model.to(device)
model.eval()

def predict(text):
    # Tokenize the input text
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Run inference
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        prediction = torch.argmax(output, dim=1).item()

    # Map prediction to label (0 or 1 for binary classification in IMDB dataset)
    label_map = {0: "Negative", 1: "Positive"}
    return label_map[prediction]

# Example usage
text = "The actors did amazing acting"
print("Prediction:", predict(text))


Prediction: Positive
